# Various performance tips and helpful tools

## Code inspection macros

We can inspect the steps in the compilation process of a given function call all the way down to LLVM, thanks to a couple of macros. These can come in really handy when debugging performance issues.

In [1]:
function foo(x)
    x^2 + 3x - 1
end

foo (generic function with 1 method)

In [2]:
foo(1)

3

In [3]:
@code_lowered foo(1)

CodeInfo(
1 ─ %1 = Main.:^
│   %2 = Core.apply_type(Base.Val, 2)
│   %3 = (%2)()
│   %4 = Base.literal_pow(%1, x, %3)
│   %5 = 3 * x
│   %6 = %4 + %5
│   %7 = %6 - 1
└──      return %7
)

In [4]:
@code_typed foo(1)

CodeInfo(
1 ─ %1 = Base.mul_int(x, x)::Int64
│   %2 = Base.mul_int(3, x)::Int64
│   %3 = Base.add_int(%1, %2)::Int64
│   %4 = Base.sub_int(%3, 1)::Int64
└──      return %4
) => Int64

In [5]:
@code_warntype foo(1)

MethodInstance for foo(::Int64)
  from foo(x) @ Main ~/Documents/Talks/CECI-Julia-for-HPC/code/performance.ipynb:1
Arguments
  #self#::Core.Const(foo)


  x::Int64
Body::Int64


1 ─ %1 = Main.:^::Core.Const(^)


│   %2 = Core

.apply_type(Base.Val, 2)::Core.Const(Val{2})
│   %3 = (%2)()::Core.Const(Val{2}())
│   %4 = Base.literal_pow(%1, x, %3)::Int64
│   %5 = (3 * x)::Int64
│   %6 = (%4 + %5)::Int64
│   %7 = (%6 - 1)::Int64
└──      return %

7



In [6]:
@code_llvm foo(1)

;  @ /home/csimal/Documents/Talks/CECI-Julia-for-HPC/code/performance.ipynb:1 within `foo`
define i64 @julia_foo_2112(i64 signext %0) #0 {
top:
;  @ /home/csimal/Documents/Talks/CECI-Julia-for-HPC/code/performance.ipynb:2 within `foo`
; ┌ @ int.jl:87 within `+`
   %1 

= add i64 %0, 3
   %2 = mul i64 %1, %0
; └
; ┌ @ int.jl:86 within `-`
   %3 = add i64 %2, -1
; └
  ret i64 %3
}


## Array views
When modifying arrays slice by slice, it's often handy to use `views` which are essentially "sub-arrays" that are directly bound to the same underlying memory, while having adapted indices.

In [16]:
function inplace!(a)
    for i in eachindex(a)
        a[i] = a[i]^2
    end
end

inplace! (generic function with 1 method)

In [13]:
A = rand(5,5)

5×5 Matrix{Float64}:
 0.286545   0.72044   0.966052   0.436461  0.422968
 0.0746006  0.125015  0.587949   0.660503  0.915845
 0.988985   0.142092  0.752617   0.148756  0.463857
 0.453155   0.880103  0.0177331  0.938454  0.699344
 0.317712   0.76428   0.534678   0.600175  0.698504

In [17]:
for i in axes(A,1)
    inplace!(A[i,:])
end
A

5×5 Matrix{Float64}:
 0.28264    0.659715  0.822647   0.422734  0.410468
 0.0745315  0.12469   0.554655   0.613514  0.793077
 0.835468   0.141615  0.683552   0.148208  0.447401
 0.437805   0.770804  0.0177321  0.806646  0.643716
 0.312394   0.692017  0.509564   0.564787  0.643073

`A` hasn't changed! That's because `A[i,:]` creates a copy.

In order to get the correct behavior, we need to use views.

In [18]:
for i in axes(A,1)
    inplace!(view(A, i, :))
end
A

5×5 Matrix{Float64}:
 0.0798852   0.435224   0.676749     0.178704   0.168484
 0.00555494  0.0155476  0.307642     0.3764     0.628972
 0.698008    0.0200547  0.467243     0.0219657  0.200167
 0.191673    0.594139   0.000314429  0.650677   0.41437
 0.0975902   0.478888   0.259656     0.318984   0.413543

Equivalently, we can use the `@views` macro, which will apply to every indexing operation within a block

In [19]:
 @views for i in axes(A,1)
    inplace!(A[i,:])
 end
 A

5×5 Matrix{Float64}:
 0.00638164  0.18942      0.457989    0.0319353    0.0283869
 3.08573e-5  0.000241728  0.0946439   0.141677     0.395606
 0.487215    0.000402191  0.218316    0.000482494  0.0400669
 0.0367385   0.353001     9.88654e-8  0.423381     0.171703
 0.00952385  0.229334     0.0674211   0.101751     0.171018

## Type Stability

A function is said to be *type-stable* if its return type can be inferred from the types of its arguments. Type stability is pretty important, as its absence forces the Julia compiler to be more conservative. Any type unstable function will in general hurt performance, so they should be avoided at all costs.

The following function is type unstable, as the type of its output depends on the *value* of its input, not just its type.

In [20]:
function type_unstable(x)
    if x < 0
        return "Negative number"
    else
        return x
    end
end

type_unstable (generic function with 1 method)

A helpful tool to hunt for type instability is the `@code_warntype` macro.

In [21]:
@code_warntype type_unstable(1)

MethodInstance for type_unstable(::Int64)
  from type_unstable(x) @ Main ~/Documents/Talks/CECI-Julia-for-HPC/code/performance.ipynb:1
Arguments
  #self#::Core.Const(type_unstable)
  x::Int64
Body::Union{Int64, String}
1 ─ %1 = (x < 0)::Bool
└──      goto #3 if not %1
2 ─      return "Negative number"
3 ─      return x



A more subtle example of type instability is when handling arithmetic expression. Julia tends to avoid implicit type conversions, which can sometimes lead to surprises. The following function is type unstable. Can you spot why?

In [22]:
relu(x) = x < 0 ? 0 : x

relu (generic function with 1 method)

In [23]:
relu(1.0), relu(-1.0)

(1.0, 0)

In [24]:
@code_warntype relu(-1.0)

MethodInstance for relu(::Float64)
  from relu(x) @ Main ~/Documents/Talks/CECI-Julia-for-HPC/code/performance.ipynb:1
Arguments
  #self#::Core.Const(relu)
  x::Float64
Body::Union{Float64, Int64}
1 ─ %1 = (x < 0)::Bool
└──      goto #3 if not %1
2 ─      return 0
3 ─      return x



The problem here is that the ternary operator returns 0 of type `Int` regardless of the type of `x`. In order to correct this, we can use the function `zero` which will return the zero value of the same type as `x`. Other functions like that include `one(x)` and `oftype(x,y)`

In [25]:
relu(x) = x < 0 ? zero(x) : x

relu (generic function with 1 method)

In [26]:
@code_warntype relu(-1.0)

MethodInstance for relu(::Float64)
  from relu(x) @ Main ~/Documents/Talks/CECI-Julia-for-HPC/code/performance.ipynb:1
Arguments
  #self#::Core.Const(relu)
  x::Float64
Body::Float64
1 ─ %1 = (x < 0)::Bool
└──      goto #3 if not %1
2 ─ %3 = Main.zero(x)::Core.Const(0.0)
└──      return %3
3 ─      return x



Another common issue with types is when incorrectly using abstract types. For example, the following type definition is particularly problematic

In [28]:
struct BadFoo
    x # this amounts to x::Any
    y::Real
    z::Vector{Integer}
end

Why is this bad? Well, because whenever a struct has abstract field types, the Julia compiler can't tell what it will hold, so it has to be ready for anything, which means using pointers to values, rather than the values themselves.

The correct way to do this is to use a parametric type, so that for any particular instance of our type, the compiler knows exactly the types of each field.

In [27]:
struct GoodFoo{T1,T2<:Real,T3<:Integer}
    x::T1
    y::T2
    z::Vector{T3}
end

As an example of how bad abstract field types (or element types can be) let's look at the generated code with `@code_llvm`.

In [29]:
mutable struct Bar{T<:AbstractFloat}
    a::T
end

In [30]:
foo(b::Bar) = m.a + 1

foo (generic function with 2 methods)

In [31]:
code_llvm(foo, Tuple{Float64})

;  @ /home/csimal/Documents/Talks/CECI-Julia-for-HPC/code/performance.ipynb:1 within `foo`
define double @julia_foo_2797(double %0) #0 {
top:
;  @ /home/csimal/Documents/Talks/CECI-Julia-for-HPC/code/performance.ipynb:2 within `foo`
; ┌ @ intfuncs.jl:319 within `literal_pow`
; │┌ @ float.jl:410 within `*`
    %1 = fmul double %0, %0
; └└
; ┌ @ promotion.jl:411 within `*` @ float.jl:410
   %2 = fmul double %0, 3.000000e+00
; └
; ┌ @ float.jl:408 within `+`
   %3 = fadd double %1, %2
; └
; ┌ @ promotion.jl:412 within `-` @ float.jl:409
   %4 = fadd double %3, -1.000000e+00
; └
  ret double %4
}


In [32]:
code_llvm(foo, Tuple{AbstractFloat})

; WARNING: This code may not match what actually runs.
;  @ /home/csimal/Documents/Talks/CECI-Julia-for-HPC/code/performance.ipynb:1 within `foo`
define nonnull {}* @julia_foo_2799({}* noundef nonnull readonly %0) #0 {
top:
  %1 = alloca [3 x {}*], align 8
  %gcframe5 = alloca [4 x {}*], align 16
  %gcframe5.sub = getelementptr inbounds [4 x {}*], [4 x {}*]* %gcframe5, i64 0, i64 0
  %.sub = getelementptr inbounds [3 x {}*], [3 x {}*]* %1, i64 0, i64 0
  %2 = bitcast [4 x {}*]* %gcframe5 to i8*
  call void @llvm.memset.p0i8.i32(i8* noundef nonnull align 16 dereferenceable(32) %2, i8 0, i32 32, i1 false)
  %thread_ptr = call i8* asm "movq %fs:0, $0", "=r"() #5
  %ppgcstack_i8 = getelementptr i8, i8* %thread_ptr, i64 -8
  %ppgcstack = bitcast i8* %ppgcstack_i8 to {}****
  %pgcstack = load {}***, {}**** %ppgcstack, align 8
;  @ /home/csimal/Documents/Talks/CECI-Julia-for-HPC/code/performance.ipynb:2 within `foo`
  %3 = bitcast [4 x {}*]* %gcframe5 to i64*
  store i64 8, i64* %3, align 16


*), {}** %8, align 8
  %22 = call nonnull {}* @ijl_apply_generic({}* inttoptr (i64 140660849103008 to {}*), {}** nonnull %.sub, i32 2)
  %23 = load {}*, {}** %4, align 8
  %24 = bitcast {}*** %pgcstack to {}**
  store {}* %23, {}** %24, align 8
  ret {}* %22
}


Notice how the second one is so much longer? That's because of all the checks the compiler has to do because it's unsure about the type. This, by the way is how Python operates by default.